# Strategy Exploration Notebook

This notebook demonstrates how to explore and validate trading strategy ideas
before implementing them as full QuantConnect algorithms.

## Contents
1. Data Loading and Preparation
2. Technical Indicator Analysis
3. Signal Generation
4. Basic Backtesting
5. Performance Analysis

In [ ]:
# Import required libraries
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

# Set display options
pd.set_option('display.max_columns', 50)
pd.set_option('display.width', 200)
plt.style.use('seaborn-v0_8-whitegrid')

# Import custom utilities
import sys
sys.path.append('..')
from utils.calculations import (
    calculate_sharpe_ratio,
    calculate_max_drawdown,
    calculate_win_rate,
    calculate_cagr
)

## 1. Data Loading and Preparation

For this example, we'll generate synthetic price data. In practice, you would
load historical data from QuantConnect or another data source.

In [ ]:
def generate_synthetic_data(n_days: int = 1000, initial_price: float = 100.0) -> pd.DataFrame:
    """
    Generate synthetic price data for strategy testing.
    
    Args:
        n_days: Number of trading days
        initial_price: Starting price
        
    Returns:
        DataFrame with OHLCV data
    """
    np.random.seed(42)
    
    # Generate returns with mean reversion tendency
    returns = np.random.normal(0.0005, 0.02, n_days)  # ~12% annual return, 20% vol
    
    # Calculate prices
    prices = initial_price * np.cumprod(1 + returns)
    
    # Generate OHLC from close
    high = prices * (1 + np.abs(np.random.normal(0, 0.01, n_days)))
    low = prices * (1 - np.abs(np.random.normal(0, 0.01, n_days)))
    open_prices = np.roll(prices, 1)
    open_prices[0] = initial_price
    
    # Generate volume
    volume = np.random.randint(1000000, 10000000, n_days)
    
    # Create DataFrame
    dates = pd.date_range(start='2020-01-01', periods=n_days, freq='B')
    df = pd.DataFrame({
        'Open': open_prices,
        'High': high,
        'Low': low,
        'Close': prices,
        'Volume': volume
    }, index=dates)
    
    return df

# Generate data
df = generate_synthetic_data(1000)
print(f"Data shape: {df.shape}")
print(f"Date range: {df.index[0]} to {df.index[-1]}")
df.head()

In [ ]:
# Plot price history
fig, axes = plt.subplots(2, 1, figsize=(14, 8), gridspec_kw={'height_ratios': [3, 1]})

# Price chart
axes[0].plot(df.index, df['Close'], label='Close Price', color='blue')
axes[0].set_title('Synthetic Price Data')
axes[0].set_ylabel('Price')
axes[0].legend()

# Volume chart
axes[1].bar(df.index, df['Volume'], color='gray', alpha=0.5)
axes[1].set_ylabel('Volume')
axes[1].set_xlabel('Date')

plt.tight_layout()
plt.show()

## 2. Technical Indicator Analysis

Let's calculate some common technical indicators to analyze the price data.

In [ ]:
def calculate_rsi(prices: pd.Series, period: int = 14) -> pd.Series:
    """Calculate RSI indicator."""
    delta = prices.diff()
    gain = (delta.where(delta > 0, 0)).rolling(window=period).mean()
    loss = (-delta.where(delta < 0, 0)).rolling(window=period).mean()
    rs = gain / loss
    return 100 - (100 / (1 + rs))

def calculate_sma(prices: pd.Series, period: int) -> pd.Series:
    """Calculate Simple Moving Average."""
    return prices.rolling(window=period).mean()

def calculate_ema(prices: pd.Series, period: int) -> pd.Series:
    """Calculate Exponential Moving Average."""
    return prices.ewm(span=period, adjust=False).mean()

def calculate_bollinger_bands(prices: pd.Series, period: int = 20, num_std: float = 2.0):
    """Calculate Bollinger Bands."""
    sma = prices.rolling(window=period).mean()
    std = prices.rolling(window=period).std()
    upper = sma + (std * num_std)
    lower = sma - (std * num_std)
    return sma, upper, lower

# Calculate indicators
df['RSI'] = calculate_rsi(df['Close'])
df['SMA_20'] = calculate_sma(df['Close'], 20)
df['SMA_50'] = calculate_sma(df['Close'], 50)
df['EMA_20'] = calculate_ema(df['Close'], 20)
df['BB_Mid'], df['BB_Upper'], df['BB_Lower'] = calculate_bollinger_bands(df['Close'])

df[['Close', 'RSI', 'SMA_20', 'SMA_50']].tail(10)

In [ ]:
# Visualize indicators
fig, axes = plt.subplots(3, 1, figsize=(14, 10), sharex=True)

# Price with moving averages
axes[0].plot(df.index, df['Close'], label='Close', alpha=0.7)
axes[0].plot(df.index, df['SMA_20'], label='SMA 20', linestyle='--')
axes[0].plot(df.index, df['SMA_50'], label='SMA 50', linestyle='--')
axes[0].fill_between(df.index, df['BB_Lower'], df['BB_Upper'], alpha=0.2, label='Bollinger Bands')
axes[0].set_title('Price with Moving Averages')
axes[0].legend(loc='upper left')
axes[0].set_ylabel('Price')

# RSI
axes[1].plot(df.index, df['RSI'], label='RSI', color='purple')
axes[1].axhline(y=70, color='r', linestyle='--', alpha=0.5)
axes[1].axhline(y=30, color='g', linestyle='--', alpha=0.5)
axes[1].fill_between(df.index, 30, 70, alpha=0.1)
axes[1].set_title('RSI (14)')
axes[1].set_ylabel('RSI')
axes[1].set_ylim(0, 100)

# Returns distribution
returns = df['Close'].pct_change().dropna()
axes[2].hist(returns, bins=50, alpha=0.7, edgecolor='black')
axes[2].axvline(returns.mean(), color='r', linestyle='--', label=f'Mean: {returns.mean():.4f}')
axes[2].set_title('Daily Returns Distribution')
axes[2].set_xlabel('Return')
axes[2].legend()

plt.tight_layout()
plt.show()

## 3. Signal Generation

Let's implement a simple RSI-based strategy to generate trading signals.

In [ ]:
def generate_rsi_signals(df: pd.DataFrame, oversold: int = 30, overbought: int = 70) -> pd.DataFrame:
    """
    Generate trading signals based on RSI.
    
    Buy when RSI crosses above oversold threshold.
    Sell when RSI crosses above overbought threshold.
    """
    signals = df.copy()
    
    # Create signal column
    signals['Signal'] = 0
    
    # Buy signal: RSI crosses above oversold
    signals.loc[(signals['RSI'] < oversold) & (signals['RSI'].shift(1) >= oversold), 'Signal'] = 1
    signals.loc[signals['RSI'] < oversold, 'Signal'] = 1  # Simplified: buy when RSI < 30
    
    # Sell signal: RSI crosses above overbought  
    signals.loc[signals['RSI'] > overbought, 'Signal'] = -1
    
    # Create position (hold until opposite signal)
    signals['Position'] = 0
    position = 0
    for i in range(len(signals)):
        if signals.iloc[i]['Signal'] == 1 and position == 0:
            position = 1
        elif signals.iloc[i]['Signal'] == -1 and position == 1:
            position = 0
        signals.iloc[i, signals.columns.get_loc('Position')] = position
    
    return signals

# Generate signals
signals = generate_rsi_signals(df)

# Count signals
buy_signals = (signals['Signal'] == 1).sum()
sell_signals = (signals['Signal'] == -1).sum()
print(f"Buy signals: {buy_signals}")
print(f"Sell signals: {sell_signals}")
print(f"Days in position: {signals['Position'].sum()} / {len(signals)}")

In [ ]:
# Visualize signals
fig, ax = plt.subplots(figsize=(14, 6))

ax.plot(signals.index, signals['Close'], label='Close', alpha=0.7)

# Mark buy signals
buy_mask = signals['Signal'] == 1
ax.scatter(signals.index[buy_mask], signals['Close'][buy_mask], 
           marker='^', color='green', s=100, label='Buy Signal')

# Mark sell signals
sell_mask = signals['Signal'] == -1
ax.scatter(signals.index[sell_mask], signals['Close'][sell_mask],
           marker='v', color='red', s=100, label='Sell Signal')

# Highlight position periods
position_periods = signals['Position'] == 1
ax.fill_between(signals.index, signals['Close'].min(), signals['Close'].max(),
                where=position_periods, alpha=0.2, color='green', label='In Position')

ax.set_title('RSI Strategy Signals')
ax.set_xlabel('Date')
ax.set_ylabel('Price')
ax.legend()
plt.show()

## 4. Basic Backtesting

Calculate strategy returns and compare to buy-and-hold.

In [ ]:
def backtest_strategy(signals: pd.DataFrame, initial_capital: float = 100000) -> pd.DataFrame:
    """
    Backtest the strategy and calculate portfolio value over time.
    """
    results = signals.copy()
    
    # Calculate daily returns
    results['Daily_Return'] = results['Close'].pct_change()
    
    # Strategy returns (only when in position)
    results['Strategy_Return'] = results['Daily_Return'] * results['Position'].shift(1)
    
    # Cumulative returns
    results['Cumulative_Market'] = (1 + results['Daily_Return']).cumprod()
    results['Cumulative_Strategy'] = (1 + results['Strategy_Return']).cumprod()
    
    # Portfolio values
    results['Portfolio_Market'] = initial_capital * results['Cumulative_Market']
    results['Portfolio_Strategy'] = initial_capital * results['Cumulative_Strategy']
    
    return results

# Run backtest
results = backtest_strategy(signals)

# Display final values
print(f"Initial Capital: ${100000:,.2f}")
print(f"Final Portfolio (Buy & Hold): ${results['Portfolio_Market'].iloc[-1]:,.2f}")
print(f"Final Portfolio (Strategy): ${results['Portfolio_Strategy'].iloc[-1]:,.2f}")

In [ ]:
# Plot equity curves
fig, ax = plt.subplots(figsize=(14, 6))

ax.plot(results.index, results['Portfolio_Market'], label='Buy & Hold', alpha=0.7)
ax.plot(results.index, results['Portfolio_Strategy'], label='RSI Strategy', alpha=0.7)

ax.set_title('Equity Curve Comparison')
ax.set_xlabel('Date')
ax.set_ylabel('Portfolio Value ($)')
ax.legend()
ax.grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

## 5. Performance Analysis

Calculate key performance metrics for the strategy.

In [ ]:
def calculate_performance_metrics(results: pd.DataFrame) -> dict:
    """Calculate comprehensive performance metrics."""
    # Get returns
    strategy_returns = results['Strategy_Return'].dropna().tolist()
    market_returns = results['Daily_Return'].dropna().tolist()
    
    # Equity curves
    strategy_equity = results['Portfolio_Strategy'].dropna().tolist()
    market_equity = results['Portfolio_Market'].dropna().tolist()
    
    # Years for CAGR
    years = len(results) / 252
    
    metrics = {
        'Strategy': {
            'Total Return': (strategy_equity[-1] / strategy_equity[0] - 1) * 100,
            'CAGR': calculate_cagr(strategy_equity[0], strategy_equity[-1], years) * 100 if calculate_cagr(strategy_equity[0], strategy_equity[-1], years) else 0,
            'Sharpe Ratio': calculate_sharpe_ratio(strategy_returns) or 0,
            'Max Drawdown': calculate_max_drawdown(strategy_equity)[0] * 100,
            'Volatility': np.std(strategy_returns) * np.sqrt(252) * 100,
        },
        'Buy & Hold': {
            'Total Return': (market_equity[-1] / market_equity[0] - 1) * 100,
            'CAGR': calculate_cagr(market_equity[0], market_equity[-1], years) * 100 if calculate_cagr(market_equity[0], market_equity[-1], years) else 0,
            'Sharpe Ratio': calculate_sharpe_ratio(market_returns) or 0,
            'Max Drawdown': calculate_max_drawdown(market_equity)[0] * 100,
            'Volatility': np.std(market_returns) * np.sqrt(252) * 100,
        }
    }
    
    return metrics

# Calculate metrics
metrics = calculate_performance_metrics(results)

# Display as DataFrame
metrics_df = pd.DataFrame(metrics).T
metrics_df = metrics_df.round(2)
print("\nPerformance Metrics:")
print("="*60)
print(metrics_df.to_string())

In [ ]:
# Visualize drawdowns
def calculate_drawdown_series(equity: pd.Series) -> pd.Series:
    """Calculate drawdown at each point."""
    peak = equity.expanding().max()
    drawdown = (equity - peak) / peak
    return drawdown

fig, axes = plt.subplots(2, 1, figsize=(14, 8), sharex=True)

# Equity curves
axes[0].plot(results.index, results['Portfolio_Market'], label='Buy & Hold')
axes[0].plot(results.index, results['Portfolio_Strategy'], label='RSI Strategy')
axes[0].set_title('Portfolio Value')
axes[0].set_ylabel('Value ($)')
axes[0].legend()

# Drawdowns
dd_market = calculate_drawdown_series(results['Portfolio_Market'])
dd_strategy = calculate_drawdown_series(results['Portfolio_Strategy'])

axes[1].fill_between(results.index, dd_market, 0, alpha=0.5, label='Buy & Hold DD')
axes[1].fill_between(results.index, dd_strategy, 0, alpha=0.5, label='Strategy DD')
axes[1].set_title('Drawdown')
axes[1].set_ylabel('Drawdown (%)')
axes[1].set_xlabel('Date')
axes[1].legend()

plt.tight_layout()
plt.show()

## Next Steps

Based on this analysis, potential improvements include:

1. **Parameter Optimization**: Test different RSI periods and thresholds
2. **Additional Filters**: Add trend filters (e.g., only trade in direction of SMA)
3. **Risk Management**: Add stop-loss and position sizing rules
4. **Multi-Asset**: Test strategy across multiple securities
5. **Transaction Costs**: Include commission and slippage estimates

When ready, implement the refined strategy in `algorithms/` directory.